In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.decomposition import PCA
%matplotlib inline


**The way NBA rookie contracts work is the first two years are guaranteed, and the salary is based on the position the player was drafted. After the completion of the player's first season, until October 31st, the team is able to exercise an option to re-sign the player for the third year. The team is able to exercise another option to re-sign the player for the fourth year afterwards with the same timeline. Teams are able to offer rookie extensions to players after their third year, but the player still has to play out their rookie contract. Teams sign players to extensions for security, to avoid the risk of the player leaving. Otherwise, after the end of the fourth year of the rookie contract, the player (Restricted Free Agent (RFA)) can either sign a long term contract with his current team, sign an offer with another team and see if the current team can match, or sign a one year qualifying offer with the current team.**

**What time period of predictions make the most sense for the stakeholders (NBA teams, owners, GMs)? They want to be able to draft a player they can sign for a long time, mitigating any uncertainties for the team. They want to avoid the circumstance where another team offers a higher salary when the player is a RFA and they have to match. Therefore the goal is find out as soon as possible whether the team should give an extension to the player, however, we also have to balance the decreasing accuracy of predicting years that are further away. No point predicting 1st or 2nd year since the rookie is guaranteed under contract. 3rd year is a good choice, as remarkable results will allow the team to sign the player as early as possible, however, it has room for error such as a player having a fluke season or is a late bloomer. The team also has the leverage to exercise the 3rd year option to keep the player. The same argument can be applied for predicting the 4th year results. This brings us to year 5, where the team has no control over the player. 5 years is a generous amount of time for a player to flourish and develop his skills, and typically is either at or approaching his prime. Compounding with the fact that the team has no control over the player, year 5 seems like the optimal year for predictions, balancing team security and prediction accuracy**

In [3]:
df = pd.read_csv('/Users/mattlee/GA-Capstone-Project/nbaadv.csv')
pd.set_option('display.max_columns',50)
pd.get_option('display.max_rows')
df.head()

,Year,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1991,Alaa Abdelnaby,PF,22,POR,43,290,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-4.2,-0.7,-5.0,-0.2
1,1991,Mahmoud Abdul-Rauf,PG,21,DEN,67,1505,12.2,0.448,0.099,0.097,1.9,6.0,3.8,19.2,1.5,0.1,9.5,27.2,-0.7,-0.3,-1.0,-0.031,-1.7,-4.4,-6.1,-1.6
2,1991,Mark Acres,C,28,ORL,68,1313,9.2,0.551,0.014,0.472,11.3,18.7,14.9,2.5,0.9,1.1,14.0,9.3,1.4,1.1,2.5,0.090,-2.1,0.3,-1.8,0.1
3,1991,Michael Adams,PG,28,DEN,66,2346,22.3,0.530,0.397,0.372,2.1,8.8,5.2,39.4,2.6,0.1,12.7,28.5,5.8,0.4,6.3,0.128,7.1,-2.7,4.4,3.8
4,1991,Mark Aguirre,SF,31,DET,78,2006,16.7,0.526,0.086,0.349,7.6,13.7,10.7,11.6,1.2,0.6,10.9,25.7,2.8,2.7,5.5,0.132,0.9,-0.3,0.7,1.4


We take our rookie year CSV made in the Rookie Year notebook and will merge it into our advanced NBA metrics CSV, so we have the rookie year of the player.

In [16]:
df1 = pd.read_csv('/Users/mattlee/GA-Capstone-Project/rookieyear.csv')
df1.head()

,Player,SeasonYear,Pos,Team,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Name
0,A.C. Green,1991,PF,LAL,82,21,26.4,3.1,6.6,0.476,0.1,0.7,0.200,3.0,5.9,0.507,0.486,2.7,3.7,0.738,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,A.C. Green
1,A.J. Bramlett,2000,C,CLE,8,0,7.6,0.5,2.6,0.190,0.0,0.0,NaN,0.5,2.6,0.190,0.190,0.0,0.0,NaN,1.5,1.3,2.8,0.0,0.1,0.0,0.4,1.6,1.0,A.J. Bramlett
2,A.J. English,1991,SG,WSB,70,12,20.6,3.6,8.2,0.439,0.0,0.4,0.097,3.5,7.7,0.458,0.441,1.6,2.2,0.707,0.9,1.2,2.1,2.5,0.4,0.2,1.6,1.8,8.8,A.J. English
3,A.J. Guyton,2001,PG,CHI,33,8,19.1,2.4,5.8,0.406,0.8,2.1,0.391,1.5,3.7,0.415,0.477,0.5,0.5,0.833,0.3,0.8,1.1,1.9,0.3,0.2,0.7,1.1,6.0,A.J. Guyton
4,A.J. Hammons,2017,C,DAL,22,0,7.4,0.8,1.9,0.405,0.2,0.5,0.500,0.5,1.5,0.375,0.464,0.4,0.9,0.450,0.4,1.3,1.6,0.2,0.0,0.6,0.5,1.0,2.2,A.J. Hammons


In [17]:
df = pd.merge(df,df1[['Name','SeasonYear']], how = 'left', on = 'Name')

In [18]:
df.shape

(19029, 29)

In [22]:
targetdf = df.loc[df['SeasonYear_x'].notnull()].copy()

In [24]:
targetdf.head()
targetdf['SeasonYear'] = targetdf['SeasonYear_x']
targetdf.drop(['SeasonYear_x','SeasonYear_y'], axis = 1, inplace = True)

In [25]:
targetdf.columns

Index([u'Year', u'Name', u'Pos', u'Age', u'Tm', u'G', u'MP', u'PER', u'TS%',
       u'3PAr', u'FTr', u'ORB%', u'DRB%', u'TRB%', u'AST%', u'STL%', u'BLK%',
       u'TOV%', u'USG%', u'OWS', u'DWS', u'WS', u'WS/48', u'OBPM', u'DBPM',
       u'BPM', u'VORP', u'SeasonYear'],
      dtype='object')

In [26]:
targetdf = targetdf[['Year','SeasonYear','Name','Pos','Age','Tm','G','MP','PER','TS%','3PAr','FTr','ORB%',
                    'DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM',
                    'DBPM','BPM','VORP']]

In [29]:
targetdf['Year in NBA'] = targetdf['Year'] - targetdf['SeasonYear'] + 1

In [30]:
targetdf = targetdf[['Year','SeasonYear','Year in NBA','Name','Pos','Age','Tm','G','MP','PER','TS%','3PAr','FTr','ORB%',
                    'DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM',
                    'DBPM','BPM','VORP']]

In [31]:
targetdf.head()

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
618,1992,1992.0,1.0,Victor Alexander,C,22,GSW,80,1350,13.9,0.561,0.002,0.325,8.7,18.3,13.6,3.1,1.5,2.7,14.8,18.3,1.1,1.4,2.5,0.090,-2.2,-0.7,-2.9,-0.3
623,1992,1992.0,1.0,Kenny Anderson,PG,21,NJN,64,1086,12.8,0.430,0.027,0.204,3.6,9.0,6.2,26.5,3.0,0.5,15.6,22.9,-0.6,1.1,0.5,0.022,-2.7,-0.9,-3.6,-0.5
630,1992,1992.0,1.0,Greg Anthony,PG,24,NYK,82,1510,10.6,0.443,0.126,0.363,2.5,7.7,5.2,28.6,2.0,0.4,16.3,17.4,0.2,2.0,2.2,0.070,-2.6,-0.3,-2.8,-0.3
634,1992,1992.0,1.0,Stacey Augmon,SG,23,ATL,82,2505,14.4,0.526,0.007,0.356,7.9,9.9,8.9,11.4,2.4,0.6,14.8,20.1,1.5,2.2,3.7,0.071,-0.3,0.6,0.3,1.5
635,1992,1992.0,1.0,Isaac Austin,C,22,UTA,31,112,15.0,0.515,0.000,0.652,11.6,23.1,17.6,6.7,0.9,1.1,11.9,25.9,0.1,0.2,0.3,0.112,-2.8,-2.4,-5.2,-0.1


**We need to include eFG% from our other dataframe**

In [36]:
brpg = pd.read_csv('/Users/mattlee/GA-Capstone-Project/brpergame.csv')
brpg.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Year
0,1,Alaa Abdelnaby\abdelal01,PF,22,POR,43,0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,2,Mahmoud Abdul-Rauf\abdulma02,PG,21,DEN,67,19,22.5,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,3,Mark Acres\acresma01,C,28,ORL,68,0,19.3,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,4,Michael Adams\adamsmi01,PG,28,DEN,66,66,35.5,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,5,Mark Aguirre\aguirma01,SF,31,DET,78,13,25.7,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [37]:
brpg['Name'] = brpg['Player'].apply(lambda x: x.split('\\')[0].lower().title())
brpg['Name'] = brpg['Name'].apply(lambda x: x.replace('*',''))

In [46]:
brpg['Team'] = brpg['Tm']

In [49]:
target = pd.merge(targetdf,brpg[['Name','eFG%','Year','Tm']], how = 'left', on = ['Name','Tm','Year'])

In [50]:
target.head()

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
0,1992,1992.0,1.0,Victor Alexander,C,22,GSW,80,1350,13.9,0.561,0.002,0.325,8.7,18.3,13.6,3.1,1.5,2.7,14.8,18.3,1.1,1.4,2.5,0.090,-2.2,-0.7,-2.9,-0.3,0.529
1,1992,1992.0,1.0,Kenny Anderson,PG,21,NJN,64,1086,12.8,0.430,0.027,0.204,3.6,9.0,6.2,26.5,3.0,0.5,15.6,22.9,-0.6,1.1,0.5,0.022,-2.7,-0.9,-3.6,-0.5,0.393
2,1992,1992.0,1.0,Greg Anthony,PG,24,NYK,82,1510,10.6,0.443,0.126,0.363,2.5,7.7,5.2,28.6,2.0,0.4,16.3,17.4,0.2,2.0,2.2,0.070,-2.6,-0.3,-2.8,-0.3,0.379
3,1992,1992.0,1.0,Stacey Augmon,SG,23,ATL,82,2505,14.4,0.526,0.007,0.356,7.9,9.9,8.9,11.4,2.4,0.6,14.8,20.1,1.5,2.2,3.7,0.071,-0.3,0.6,0.3,1.5,0.490
4,1992,1992.0,1.0,Isaac Austin,C,22,UTA,31,112,15.0,0.515,0.000,0.652,11.6,23.1,17.6,6.7,0.9,1.1,11.9,25.9,0.1,0.2,0.3,0.112,-2.8,-2.4,-5.2,-0.1,0.457


In [869]:
target.loc[(target['Name'] == 'Derrick Rose') & (target['Year in NBA'] == 6.0), 'Year in NBA'] = 5.0

**We have the option to take the average of the five year period or take the fifth year statistics.**

These are duplicates as a result of players being traded, which results in multiple entries of the player, and merging only on name and year. 

In [280]:
target.loc[target['Tm'] == 'TOT']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
29,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
30,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
31,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
32,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
33,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
34,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
35,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
36,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
37,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384
57,1992,1992.0,1.0,Mark Randall,PF,24,TOT,54,441,12.3,0.509,0.107,0.289,9.6,8.2,8.9,10.5,1.4,0.4,13.0,18.6,0.3,0.1,0.5,0.050,-1.0,-2.4,-3.4,-0.2,0.466


In [290]:
target1 = target.copy()

In [291]:
target1 = target1.drop_duplicates()

In [293]:
target1.to_csv('advancedstatsdup.csv', index = False)

In [314]:
target1.loc[target1['Tm'] == 'TOT']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%,yr5player
29,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,-5.3,-0.4,-5.7,-0.4,0.384,0
57,1992,1992.0,1.0,Mark Randall,PF,24,TOT,54,441,12.3,0.509,0.107,0.289,9.6,8.2,8.9,10.5,1.4,0.4,13.0,18.6,0.3,0.1,0.5,0.050,-1.0,-2.4,-3.4,-0.2,0.466,0
104,1993,1992.0,2.0,Chris Corchiani,PG,24,TOT,10,105,14.8,0.662,0.125,0.875,1.1,6.1,3.8,22.0,2.8,0.0,19.4,16.9,0.2,0.1,0.3,0.159,0.3,-1.3,-1.0,0.0,0.583,0
146,1993,1992.0,2.0,Mark Randall,PF,25,TOT,37,248,11.9,0.530,0.100,0.325,11.5,13.1,12.3,6.6,0.8,0.5,15.7,18.8,0.1,0.2,0.3,0.061,-2.2,-1.7,-3.9,-0.1,0.506,0
207,1994,1992.0,3.0,Pete Chilcutt,PF,25,TOT,76,1365,12.7,0.472,0.033,0.145,10.1,19.8,14.9,9.7,1.9,1.8,13.4,17.2,0.1,1.4,1.5,0.052,-2.5,0.9,-1.6,0.1,0.456,0
229,1994,1992.0,3.0,Sean Green,SG,23,TOT,36,334,7.6,0.390,0.224,0.098,3.3,7.9,5.6,8.1,2.7,1.1,12.4,28.4,-1.1,0.2,-0.9,-0.130,-6.4,-3.3,-9.7,-0.7,0.372,0
251,1994,1992.0,3.0,Luc Longley,C,25,TOT,76,1502,13.5,0.508,0.002,0.269,10.2,23.4,16.9,11.3,1.5,3.4,18.6,18.7,0.0,2.2,2.2,0.071,-2.9,1.8,-1.1,0.3,0.471,0
257,1994,1992.0,3.0,Mark Macon,SG,24,TOT,42,496,8.0,0.410,0.054,0.185,4.0,5.2,4.6,15.5,4.0,0.1,16.7,21.0,-0.9,0.4,-0.4,-0.042,-4.6,-0.3,-4.9,-0.4,0.380,0
308,1994,1992.0,3.0,Labradford Smith,SG,24,TOT,66,877,11.6,0.484,0.196,0.275,4.2,6.4,5.3,18.7,2.2,0.4,12.7,18.9,0.4,0.4,0.8,0.045,-1.3,-1.9,-3.2,-0.3,0.440,0
353,1995,1995.0,1.0,Howard Eisley,PG,22,TOT,49,552,6.1,0.430,0.303,0.328,2.7,7.8,5.3,26.4,1.7,0.8,26.4,15.6,-0.7,0.2,-0.5,-0.044,-4.9,-2.1,-7.0,-0.7,0.365,0


In [318]:
target1.groupby()

In [315]:
targettest = target1.copy()

Fifth Year Only
==

In [870]:
fifth = target.loc[target['Year in NBA'] == 5.0].copy()

In [871]:
fifth = fifth.reset_index(drop = True)
fifth.shape

(853, 30)

In [872]:
fifth.head()

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
0,1996,1992.0,5.0,Kenny Anderson,PG,25,TOT,69,2344,19.5,0.534,0.307,0.405,3.0,7.1,5.0,40.4,2.5,0.5,12.9,21.4,6.1,1.6,7.7,0.158,4.2,-1.9,2.3,2.6,0.474
1,1996,1992.0,5.0,Kenny Anderson,PG,25,NJN,31,1042,19.8,0.511,0.261,0.495,3.8,6.9,5.3,40.4,2.6,0.6,11.1,21.8,2.7,1.0,3.7,0.172,4.3,-1.0,3.2,1.4,0.424
2,1996,1992.0,5.0,Kenny Anderson,PG,25,CHH,38,1302,19.2,0.555,0.346,0.330,2.4,7.2,4.8,40.4,2.4,0.3,14.5,21.1,3.4,0.6,4.0,0.148,4.1,-2.5,1.6,1.2,0.515
3,1996,1992.0,5.0,Greg Anthony,PG,28,VAN,69,2096,18.6,0.530,0.347,0.380,1.6,8.4,4.9,41.9,2.9,0.4,14.9,23.3,2.9,1.9,4.8,0.111,2.9,-1.6,1.3,1.7,0.472
4,1996,1992.0,5.0,Stacey Augmon,SF,27,ATL,77,2294,15.9,0.556,0.005,0.430,6.8,9.0,7.8,10.0,2.5,1.0,13.6,19.9,3.5,2.1,5.6,0.117,-0.1,0.2,0.1,1.2,0.491


**Now we want to decide what metrics to look at**

**We have a couple goals in mind**

* Overall skill - PER, WS, WS/48, BPM, VORP
* Offensive - eFG%, TS%, FTr, Usage, ORB%
* Defensive - DRB%, DWS, DBPM


In [94]:
fifth.to_csv('metric5.csv',index = False)

In [179]:
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows', 10000)

In [264]:
fifth.loc[fifth['Name'] == 'Eric Maynor']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
587,2014,2010.0,5.0,Eric Maynor,PG,26,TOT,31,327,6.2,0.387,0.366,0.109,3.1,10.6,6.7,23.0,1.4,0.7,21.5,18.2,-0.6,0.2,-0.4,-0.058,-5.8,-1.9,-7.6,-0.5,0.376


In [263]:
fifth.loc[(fifth['Name'] == 'Terrence Ross') & (fifth['Year'] == 2017)]

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
697,2017,2013.0,5.0,Terrence Ross,SF,25,TOT,78,1955,13.4,0.55,0.524,0.103,1.0,10.9,5.9,7.1,2.2,1.4,8.0,19.3,1.5,2.0,3.5,0.086,0.3,-0.6,-0.3,0.8,0.532


**Performing some manual clean up of multiple entries of players who were traded mid-season. For example, if Eric Maynor was traded from Utah to Oklahoma, he would have three entries, one with Utah, one with Oklahoma, and one as an aggregate of that season.**

In [261]:
fifth.drop(fifth.index[[698,699]], inplace = True)
fifth = fifth.reset_index(drop = True)

In [262]:
fifth.to_csv('metric5.csv', index = False)

In [285]:
fifthdup = fifth.drop_duplicates()

In [289]:
fifthdup.to_csv('metrics5.csv', index = False)

**Attempt to include rookie year and college for easier identification**

In [863]:
fifthdup.shape

(598, 30)

In [873]:
target.loc[target['Name'] == 'Derrick Rose']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
6611,2009,2009.0,1.0,Derrick Rose,PG,20,CHI,81,3000,16.0,0.516,0.060,0.207,3.7,8.4,6.1,28.8,1.1,0.4,13.3,22.6,3.4,1.5,4.9,0.078,0.9,-1.5,-0.6,1.1,0.482
7249,2010,2009.0,2.0,Derrick Rose,PG,21,CHI,78,2871,18.6,0.532,0.044,0.246,2.6,8.7,5.7,30.3,1.0,0.7,12.5,27.2,3.5,2.5,6.0,0.100,1.7,-1.3,0.5,1.8,0.495
7854,2011,2009.0,3.0,Derrick Rose,PG,22,CHI,81,3026,23.5,0.550,0.241,0.348,3.2,9.4,6.4,38.7,1.5,1.3,13.1,32.2,8.3,4.8,13.1,0.208,5.8,0.1,5.9,6.0,0.485
8392,2012,2009.0,4.0,Derrick Rose,PG,23,CHI,39,1375,23.0,0.532,0.249,0.344,2.1,8.4,5.3,40.3,1.4,1.4,12.9,30.5,4.1,2.0,6.0,0.211,5.5,-0.6,4.9,2.4,0.473
9571,2014,2009.0,5.0,Derrick Rose,PG,25,CHI,10,311,9.7,0.446,0.287,0.195,3.3,8.3,5.9,26.2,0.9,0.2,16.0,31.5,-0.6,0.4,-0.2,-0.036,-2.8,-2.3,-5.1,-0.2,0.402
10191,2015,2009.0,7.0,Derrick Rose,PG,26,CHI,51,1530,15.9,0.493,0.325,0.224,2.6,8.7,5.7,30.7,1.2,0.8,14.9,31.7,0.0,1.2,1.2,0.038,1.1,-2.3,-1.3,0.3,0.450
10750,2016,2009.0,8.0,Derrick Rose,PG,27,CHI,66,2097,13.5,0.479,0.143,0.171,2.4,8.8,5.6,25.5,1.1,0.5,13.4,27.3,-0.8,1.2,0.4,0.009,-1.2,-2.1,-3.3,-0.7,0.448
11310,2017,2009.0,9.0,Derrick Rose,PG,28,NYK,64,2082,17.0,0.530,0.061,0.259,3.4,9.3,6.4,22.8,1.1,0.7,11.9,25.7,2.4,0.6,3.0,0.068,0.2,-2.2,-1.9,0.0,0.477


In [875]:
drose = target.loc[(target['Name'] == 'Derrick Rose') & (target['Year in NBA'] == 5.0)].copy()

In [876]:
fifthdup.tail()

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
697,2017,2013.0,5.0,Terrence Ross,SF,25,TOT,78,1955,13.4,0.550,0.524,0.103,1.0,10.9,5.9,7.1,2.2,1.4,8.0,19.3,1.5,2.0,3.5,0.086,0.3,-0.6,-0.3,0.8,0.532
698,2017,2013.0,5.0,Mike Scott,PF,28,ATL,18,195,6.1,0.366,0.466,0.138,5.7,15.0,10.4,12.5,1.0,1.3,11.5,15.6,-0.3,0.2,0.0,-0.009,-5.1,0.0,-5.1,-0.2,0.328
699,2017,2013.0,5.0,Jared Sullinger,PF,24,TOR,11,118,5.6,0.359,0.375,0.167,9.6,16.3,12.9,3.7,1.7,0.7,7.2,21.0,-0.2,0.1,-0.1,-0.040,-5.2,-2.6,-7.8,-0.2,0.344
700,2017,2013.0,5.0,Dion Waiters,SG,25,MIA,46,1384,14.5,0.506,0.325,0.191,1.4,11.0,6.2,23.8,1.5,1.1,12.5,26.4,0.2,1.5,1.6,0.056,-0.1,-0.8,-0.9,0.4,0.487
701,2017,2013.0,5.0,Tyler Zeller,C,27,BOS,51,525,13.0,0.508,0.006,0.247,9.2,17.0,13.2,12.2,0.7,3.3,10.2,16.5,0.5,0.6,1.0,0.094,-3.2,0.8,-2.5,-0.1,0.494


In [878]:
fifthdup1 = pd.concat([fifthdup,drose])

In [881]:
fifthdup1 = fifthdup1.reset_index(drop = True)

In [882]:
fifthdup1.to_csv('metrics5edit.csv', index = False)

**Just making sure Derrick Rose is in there. I only included players who had played 5 straight years. Derrick Rose was injured for his entire 5th year, so he skipped from 4 to 6 and was omitted.**

In [889]:
drose.to_csv('drose5.csv', index = False)

5 Year Average
==

In [272]:
names = list(fifth['Name'].unique())

In [329]:
fiveavg = target1.copy()

In [331]:
fiveavg = fiveavg.loc[fiveavg['Name'].isin(names)].copy()

In [400]:
five = fiveavg[fiveavg['Year in NBA'] <= 5.0].copy()

In [389]:
five.loc[five['Tm'] == 'TOT']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,eFG%,yr5player,Mean_G,Mean_MP,Mean_PER,Mean_TS%,Mean_3PAr,Mean_FTr,Mean_ORB%,Mean_DRB%,Mean_TRB%,Mean_AST%,Mean_STL%,Mean_BLK%,Mean_TOV%,Mean_USG%,Mean_OWS,Mean_DWS,Mean_WS,Mean_WS/48,Mean_OBPM,Mean_DBPM,Mean_BPM,Mean_VORP,Mean_eFG%
29,1992,1992.0,1.0,Elliot Perry,PG,22,TOT,50,437,8.6,0.428,0.054,0.318,3.4,6.3,4.8,22.6,3.7,0.4,25.4,18.4,-0.8,0.4,-0.4,-0.046,...,0.384,0,63.800000,2355.400000,20.020000,0.553600,0.217800,0.396600,4.740000,9.600000,7.140000,27.240000,2.640000,1.120000,14.560000,24.760000,29.1,12.2,41.4,0.155600,3.640000,0.200000,3.880000,19.5,0.497200
207,1994,1992.0,3.0,Pete Chilcutt,PF,25,TOT,76,1365,12.7,0.472,0.033,0.145,10.1,19.8,14.9,9.7,1.9,1.8,13.4,17.2,0.1,1.4,1.5,0.052,...,0.456,0,49.200000,1241.600000,13.220000,0.510600,0.066800,0.577800,5.880000,9.080000,7.420000,10.120000,1.660000,0.500000,12.200000,21.320000,5.7,1.7,7.3,0.051800,-1.320000,-2.200000,-3.540000,-1.9,0.426600
251,1994,1992.0,3.0,Luc Longley,C,25,TOT,76,1502,13.5,0.508,0.002,0.269,10.2,23.4,16.9,11.3,1.5,3.4,18.6,18.7,0.0,2.2,2.2,0.071,...,0.471,0,48.800000,791.200000,10.280000,0.560800,0.000000,0.592200,10.380000,17.260000,13.820000,5.180000,1.100000,2.100000,20.240000,12.960000,5.5,3.3,9.1,0.087400,-2.580000,0.260000,-2.340000,0.6,0.504600
257,1994,1992.0,3.0,Mark Macon,SG,24,TOT,42,496,8.0,0.410,0.054,0.185,4.0,5.2,4.6,15.5,4.0,0.1,16.7,21.0,-0.9,0.4,-0.4,-0.042,...,0.380,0,14.400000,126.200000,14.140000,0.448400,0.014200,0.487400,15.020000,27.960000,21.400000,10.840000,1.760000,3.540000,13.680000,11.200000,0.5,0.9,1.3,0.108600,-2.740000,2.600000,-0.140000,0.0,0.457400
353,1995,1995.0,1.0,Howard Eisley,PG,22,TOT,49,552,6.1,0.430,0.303,0.328,2.7,7.8,5.3,26.4,1.7,0.8,26.4,15.6,-0.7,0.2,-0.5,-0.044,...,0.365,0,65.000000,1543.000000,12.520000,0.507000,0.003200,0.511200,9.100000,18.600000,13.800000,7.040000,1.240000,2.100000,16.000000,17.140000,4.6,7.8,12.4,0.068400,-2.760000,0.200000,-2.560000,-0.2,0.461800
387,1995,1993.0,3.0,Tom Gugliotta,SF-PF,25,TOT,77,2568,15.2,0.515,0.222,0.301,7.3,18.5,12.9,18.0,2.6,1.7,16.6,19.4,0.8,2.7,3.5,0.065,...,0.479,0,57.000000,824.400000,14.300000,0.511600,0.019400,0.521600,13.400000,16.960000,15.260000,7.560000,2.640000,2.220000,18.260000,20.860000,2.5,8.2,10.8,0.119800,-1.740000,1.840000,0.140000,2.4,0.455000
416,1995,1995.0,1.0,Donyell Marshall,SF,21,TOT,72,2086,12.5,0.465,0.277,0.253,7.6,15.0,11.3,8.6,1.1,3.0,10.6,22.9,-0.9,1.2,0.3,0.007,...,0.433,0,64.800000,1051.800000,10.920000,0.471000,0.002000,0.194000,7.500000,17.320000,12.340000,6.480000,1.140000,1.560000,10.920000,18.580000,1.6,5.8,7.3,0.066000,-3.320000,-0.680000,-4.000000,-2.6,0.436200
460,1995,1993.0,3.0,Tracy Murray,SF,23,TOT,54,516,10.8,0.513,0.369,0.180,4.2,8.6,6.5,5.9,1.4,0.5,12.2,24.0,0.1,0.4,0.5,0.047,...,0.483,0,55.285714,1414.571429,16.585714,0.531857,0.094714,0.443571,2.571429,11.857143,7.128571,34.585714,1.600000,0.242857,17.185714,21.157143,14.1,5.1,19.2,0.092857,0.371429,-1.657143,-1.300000,2.4,0.463286
504,1995,1992.0,4.0,Steve Smith,SG,25,TOT,80,2665,16.1,0.559,0.414,0.369,4.4,7.7,6.0,17.4,1.2,0.9,11.7,22.5,5.3,2.4,7.8,0.140,...,0.494,0,56.400000,1090.200000,15.420000,0.567000,0.001000,0.433000,12.720000,18.180000,15.500000,3.920000,1.660000,3.260000,11.580000,12.880000,10.0,9.3,19.3,0.171200,-0.700000,1.580000,0.880000,4.7,0.517200
528,1996,1992.0,5.0,Kenny Anderson,PG,25,TOT,69,2344,19.5,0.534,0.307,0.405,3.0,7.1,5.0,40.4,2.5,0.5,12.9,21.4,6.1,1.6,7.7,0.158,...,0.474,1,67.200000,2389.200000,20.460000,0.593400,0.388600,0.202400,2.100000,10.840000,6.460000,31.200000,2.360000,0.520000,15.940000,24.980000,27.9,10.1,38.1,0.150800,4.740000,-0.900000,3.820000,17.9,0.556800


In [853]:
five.loc[five['Name'] == 'Terrence Ross']

,Year,SeasonYear,Year in NBA,Name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,eFG%,yr5player,Mean_G,Mean_MP,Mean_PER,Mean_TS%,Mean_3PAr,Mean_FTr,Mean_ORB%,Mean_DRB%,Mean_TRB%,Mean_AST%,Mean_STL%,Mean_BLK%,Mean_TOV%,Mean_USG%,Mean_OWS,Mean_DWS,Mean_WS,Mean_WS/48,Mean_OBPM,Mean_DBPM,Mean_BPM,Mean_VORP,Mean_eFG%
2628,2013,2013.0,1.0,Terrence Ross,SG,21,TOR,73,1239,10.4,0.491,0.429,0.092,3.3,10.6,6.9,7.2,1.8,0.9,9.2,19.6,0.0,0.8,0.9,0.034,...,0.478,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,2014,2013.0,2.0,Terrence Ross,SG,22,TOR,81,2159,12.0,0.553,0.543,0.130,2.3,11.3,6.8,6.1,1.5,1.0,10.0,18.6,2.0,2.2,4.2,0.094,...,0.531,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2828,2015,2013.0,3.0,Terrence Ross,SF,23,TOR,82,2092,11.2,0.519,0.518,0.074,1.8,10.7,6.2,6.5,1.3,1.0,8.1,18.3,1.3,1.0,2.4,0.054,...,0.507,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2894,2016,2013.0,4.0,Terrence Ross,SF,24,TOR,73,1747,12.9,0.551,0.542,0.099,1.4,10.6,6.1,5.3,1.6,1.2,6.6,18.2,2.1,1.6,3.6,0.100,...,0.536,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2929,2017,2013.0,5.0,Terrence Ross,SF,25,TOT,78,1955,13.4,0.550,0.524,0.103,1.0,10.9,5.9,7.1,2.2,1.4,8.0,19.3,1.5,2.0,3.5,0.086,...,0.532,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2930,2017,2013.0,5.0,Terrence Ross,SF,25,TOR,54,1207,14.9,0.558,0.534,0.104,1.3,11.7,6.5,5.9,2.2,1.4,6.2,19.8,1.6,1.3,2.9,0.116,...,0.541,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2931,2017,2013.0,5.0,Terrence Ross,SF,25,ORL,24,748,11.1,0.536,0.506,0.101,0.6,9.6,4.9,9.0,2.3,1.4,11.2,18.5,0.0,0.6,0.6,0.037,...,0.517,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [854]:
five.drop(five.index[[2930,2931]], inplace = True)
five = five.reset_index(drop = True)

In [855]:
five.to_csv('metric5dup.csv',index = False)

**We get the aggregate stats by performing groupbys.**

In [856]:
new = five.groupby('Name', as_index= False)['G'].mean()

In [857]:
new1 = pd.concat([new,
five.groupby('Name', as_index= False)['MP'].mean()['MP'],
five.groupby('Name', as_index= False)['PER'].mean()['PER'],
five.groupby('Name', as_index= False)['TS%'].mean()['TS%'],
five.groupby('Name', as_index= False)['3PAr'].mean()['3PAr'],
five.groupby('Name', as_index= False)['FTr'].mean()['FTr'],
five.groupby('Name', as_index= False)['ORB%'].mean()['ORB%'],
five.groupby('Name', as_index= False)['DRB%'].mean()['DRB%'],
five.groupby('Name', as_index= False)['TRB%'].mean()['TRB%'],
five.groupby('Name', as_index= False)['AST%'].mean()['AST%'],
five.groupby('Name', as_index= False)['STL%'].mean()['STL%'],
five.groupby('Name', as_index= False)['BLK%'].mean()['BLK%'],
five.groupby('Name', as_index= False)['TOV%'].mean()['TOV%'],
five.groupby('Name', as_index= False)['USG%'].mean()['USG%'],
five.groupby('Name', as_index= False)['OWS'].sum()['OWS'],
five.groupby('Name', as_index= False)['DWS'].sum()['DWS'],
five.groupby('Name', as_index= False)['WS'].sum()['WS'],
five.groupby('Name', as_index= False)['WS/48'].mean()['WS/48'],
five.groupby('Name', as_index= False)['OBPM'].mean()['OBPM'],
five.groupby('Name', as_index= False)['DBPM'].mean()['DBPM'],
five.groupby('Name', as_index= False)['BPM'].mean()['BPM'],
five.groupby('Name', as_index= False)['VORP'].sum()['VORP'],
five.groupby('Name', as_index= False)['eFG%'].mean()['eFG%']], axis = 1
)

In [858]:
new1.head()

,Name,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
0,A.J. Price,47.0,721.0,11.66,0.4816,0.4852,0.1718,1.84,9.06,5.40,23.00,1.50,0.16,13.28,20.86,1.1,3.2,4.4,0.0440,-0.66,-2.32,-2.98,-0.3,0.4562
1,Aaron Gray,47.8,596.6,12.82,0.5360,0.0022,0.3628,13.64,23.52,18.56,8.36,1.38,2.02,19.14,15.90,1.5,4.1,5.5,0.0952,-2.34,1.00,-1.32,0.3,0.5168
2,Aaron Mckie,68.0,1496.6,11.76,0.4844,0.1694,0.2526,3.80,12.56,8.14,15.76,2.70,0.78,15.48,15.44,3.1,10.5,13.8,0.0898,-1.60,1.54,-0.04,3.7,0.4442
3,Adam Keefe,72.8,1241.0,13.24,0.5694,0.0042,0.5914,12.16,18.62,15.42,5.48,1.58,1.26,16.96,15.18,7.9,7.8,15.7,0.1190,-1.04,0.50,-0.54,2.9,0.5122
4,Adonal Foyle,62.4,1173.2,11.74,0.4470,0.0012,0.2892,11.30,18.60,14.80,4.34,1.08,6.34,16.78,13.94,-1.4,6.2,4.8,0.0388,-4.54,2.48,-2.08,0.2,0.4408


In [859]:
new1.to_csv('avg5metrics.csv',index = False)

In [862]:
new1.shape

(598, 24)

**Again, making sure Derrick Rose is in there.**

In [884]:
droseavg = target.loc[(target['Name'] == 'Derrick Rose') & (target['Year in NBA'] < 6)].copy()

In [885]:
drosenew = droseavg.groupby('Name', as_index = False)['G'].mean()
drosenew1 = pd.concat([drosenew,
droseavg.groupby('Name', as_index= False)['MP'].mean()['MP'],
droseavg.groupby('Name', as_index= False)['PER'].mean()['PER'],
droseavg.groupby('Name', as_index= False)['TS%'].mean()['TS%'],
droseavg.groupby('Name', as_index= False)['3PAr'].mean()['3PAr'],
droseavg.groupby('Name', as_index= False)['FTr'].mean()['FTr'],
droseavg.groupby('Name', as_index= False)['ORB%'].mean()['ORB%'],
droseavg.groupby('Name', as_index= False)['DRB%'].mean()['DRB%'],
droseavg.groupby('Name', as_index= False)['TRB%'].mean()['TRB%'],
droseavg.groupby('Name', as_index= False)['AST%'].mean()['AST%'],
droseavg.groupby('Name', as_index= False)['STL%'].mean()['STL%'],
droseavg.groupby('Name', as_index= False)['BLK%'].mean()['BLK%'],
droseavg.groupby('Name', as_index= False)['TOV%'].mean()['TOV%'],
droseavg.groupby('Name', as_index= False)['USG%'].mean()['USG%'],
droseavg.groupby('Name', as_index= False)['OWS'].sum()['OWS'],
droseavg.groupby('Name', as_index= False)['DWS'].sum()['DWS'],
droseavg.groupby('Name', as_index= False)['WS'].sum()['WS'],
droseavg.groupby('Name', as_index= False)['WS/48'].mean()['WS/48'],
droseavg.groupby('Name', as_index= False)['OBPM'].mean()['OBPM'],
droseavg.groupby('Name', as_index= False)['DBPM'].mean()['DBPM'],
droseavg.groupby('Name', as_index= False)['BPM'].mean()['BPM'],
droseavg.groupby('Name', as_index= False)['VORP'].sum()['VORP'],
droseavg.groupby('Name', as_index= False)['eFG%'].mean()['eFG%']], axis = 1)

In [887]:
new2 = pd.concat([new1,drosenew1])
new2 = new2.reset_index(drop = True)
new2.to_csv('avg5metricsedit.csv',index = False)

In [888]:
drosenew1.to_csv('droseavg5.csv',index = False)

In [895]:
new2.tail(1)

,Name,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,eFG%
598,Derrick Rose,57.8,2116.6,18.16,0.5152,0.1762,0.268,2.98,8.64,5.88,32.86,1.18,0.8,13.56,28.8,18.7,11.2,29.8,0.1122,2.22,-1.12,1.12,11.1,0.4674
